In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

### Graphic libraries
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
df_train =pd.read_csv("../input/tabular-playground-series-jan-2021/train.csv")

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train.isna().sum()

In [ ]:
df_train.describe()

Now let's check test data.

In [ ]:
df_test=pd.read_csv("../input/tabular-playground-series-jan-2021/test.csv")

In [ ]:
df_test.head()

In [ ]:
df_test.isnull().sum()


In [ ]:
df_train.shape

## EDA



In [ ]:
df_train["target"].hist(bins=40, edgecolor='b', linewidth=1.0,
              xlabelsize=8, ylabelsize=8, grid=False, figsize=(6,2), color='red')    
plt.tight_layout(rect=(0, 0, 1.2, 1.2))   
plt.suptitle('Target Plot', x=0.65, y=1.25, fontsize=14); 

In [ ]:
sns.boxplot('target', data = df_train)

In [ ]:
sns.set_style("white")
sns.set_color_codes(palette='deep')
f, ax = plt.subplots(figsize=(8, 7))
#Check the new distribution 
sns.distplot(df_train['target'], color="b");
ax.xaxis.grid(False)
ax.set(ylabel="Frequency")
ax.set(xlabel="Target")
ax.set(title="Target distribution")
sns.despine(trim=True, left=True)
plt.show()

Correlations

In [ ]:
#data = pd.concat([_train['target'], train['OverallQual']], axis=1)
correlation_train=df_train.corr()
sns.set(font_scale=2)
plt.figure(figsize = (50,35))
ax = sns.heatmap(correlation_train, annot=True,annot_kws={"size": 25},fmt='.1f',cmap='PiYG', linewidths=.5)

In [ ]:
#plotting correlations
num_feat=df_train.columns[df_train.dtypes!=object]
num_feat=num_feat[1:-1] 
labels = []
values = []
for col in num_feat:
    labels.append(col)
    values.append(np.corrcoef(df_train[col].values, df_train.target.values)[0,1])
    
ind = np.arange(len(labels))
width = 0.9
fig, ax = plt.subplots(figsize=(12,40))
rects = ax.barh(ind, np.array(values), color='red')
ax.set_yticks(ind+((width)/2.))
ax.set_yticklabels(labels, rotation='horizontal')
ax.set_xlabel("Correlation coefficient")
ax.set_title("Correlation Coefficients w.r.t TArget");

In [ ]:
df_train.columns

Data is clean with continues target variable. let's make split into 70:30 train and validation data. and drop id column. 

In [ ]:
x = df_train.iloc[:, 1:15].values  
print(x) 
y = df_train.iloc[:, -1].values 

In [ ]:
# check lightgbm version
import lightgbm
from lightgbm import LGBMRegressor


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x
                                                    , y, test_size=0.10, random_state=0)

In [ ]:
LGB = LGBMRegressor(max_depth=6, learning_rate=0.15, colsample_bytree=0.8, n_estimators=600,reg_alpha=0.001,reg_lambda=0.000001,n_jobs=2,min_child_weight=5)
LGB.fit(X_train,y_train)

In [ ]:
y_pred = LGB.predict(X_test)

In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
preds=LGB.predict(df_test.iloc[:,1:].values)

In [ ]:
sub=pd.read_csv("../input/tabular-playground-series-jan-2021/sample_submission.csv")

In [ ]:
sub

In [ ]:
sub.target = preds

In [ ]:
sub.to_csv("submission.csv", index=False)